In [84]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [85]:
pip install psycopg2 sqlalchemy pandas

In [86]:
import pandas as pd
from sqlalchemy import create_engine
from psycopg2 import sql

In [87]:
# Database connection parameters
db_params = {
    "dbname": "postgres",  # Connect to the default database to create a new one
    "user": "postgres",
    "password": "1234567890",
    "host": "localhost",
    "port": 5432
}

In [88]:
# Define the SQL commands for database and tables
create_database_command = "CREATE DATABASE coursera_database;"

In [89]:
create_tables_command ="""
-- Create courses_dim table
CREATE TABLE public.courses_dim
(
    course_id INT PRIMARY KEY,
    title TEXT,
    organization TEXT
);

-- Create skills_dim table
CREATE TABLE public.skills_dim
(
    skill_id INT PRIMARY KEY,
    skills TEXT,
    type TEXT
);

-- Create course_details_fact table
CREATE TABLE public.course_details_fact
(
    course_id INT PRIMARY KEY,
    certificate_type TEXT,
    rating INT,
    reviews TEXT,
    coursera_plus BOOLEAN,
    degree_points BOOLEAN,
    difficulty TEXT
);

-- Create course_skill_dim table
CREATE TABLE public.course_skill_dim
(
    id INT PRIMARY KEY,
    skill_id INT,
    course_id INT
);

-- Set ownership
ALTER TABLE public.courses_dim OWNER TO postgres;
ALTER TABLE public.skills_dim OWNER TO postgres;
ALTER TABLE public.course_details_fact OWNER TO postgres;
ALTER TABLE public.course_skill_dim OWNER TO postgres;

-- Create indexes
CREATE INDEX idx_skill_id ON public.course_skill_dim (skill_id);
CREATE INDEX idx_course_id ON public.course_skill_dim (course_id);
"""

try:
    # Connect to PostgreSQL to create the database
    connection = psycopg2.connect(**db_params)
    connection.autocommit = True
    cursor = connection.cursor()

    # Create the Coursera database
    cursor.execute(create_database_command)
    print("Database 'coursera_database' created successfully.")

    # Connect to the newly created database
    db_params["dbname"] = "coursera_database"
    connection = psycopg2.connect(**db_params)
    cursor = connection.cursor()

    # Create tables
    cursor.execute(create_tables_command)
    connection.commit()
    print("Tables created successfully in 'coursera_database'.")

except Exception as e:
    print(f"Error: {e}")
    if connection:
        connection.rollback()
finally:
    # Close the cursor and connection
    if cursor:
        cursor.close()
    if connection:
        connection.close()

Database 'ccoursera_database' created successfully.
Tables created successfully in 'coursera_database'.


In [91]:
connection = psycopg2.connect(
    host="localhost",
    database="coursera_database",
    user="postgres",
    password="1234567890"
)

cursor = connection.cursor()
print("Connected to PostgreSQL")
cursor.close()
connection.close()

Connected to PostgreSQL


In [92]:
# Database connection details
db_params = {
    "dbname": "coursera_database",  # Connect to the default database to create a new one
    "user": "postgres",
    "password": "1234567890",
    "host": "localhost",
    "port": 5432
}

# Connection string
db_url = f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['dbname']}"

# Create SQLAlchemy engine
engine = create_engine(db_url)


# Load CSV into pandas DataFrame


In [93]:
courses_dim = pd.read_csv("data/courses_dim.csv")

In [94]:
skills_dim = pd.read_csv("data/skills_list_dim.csv")

In [95]:
course_details_fact = pd.read_csv("data/course_details_fact.csv")

In [96]:
course_to_skills_dim = pd.read_csv("data/course_to_skill_dim.csv")

# Load data into tables


In [97]:
courses_dim.to_sql('courses_dim', engine, if_exists='replace', index=False)

8

In [98]:
skills_dim.to_sql('skills_dim', engine, if_exists='replace', index=False)

715

In [99]:
course_details_fact.to_sql('course_details_fact', engine, if_exists='replace', index=False)

8

In [100]:
course_to_skills_dim.to_sql('course_to_skills_dim', engine, if_exists='replace', index=False)

344

# adding primary and foreign keys

In [107]:
# SQL script to set keys and relationships
alter_table_script = """

-- Set primary keys
ALTER TABLE courses_dim ADD CONSTRAINT courses_dim_pkey PRIMARY KEY (course_id);
ALTER TABLE skills_dim ADD CONSTRAINT skills_dim_pkey PRIMARY KEY (skill_id);
ALTER TABLE course_details_fact ADD CONSTRAINT course_details_fact_pkey PRIMARY KEY (course_id);
ALTER TABLE course_skill_dim ADD CONSTRAINT course_skill_dim_pkey PRIMARY KEY (course_id);

-- Set foreign keys
ALTER TABLE course_details_fact ADD CONSTRAINT course_details_fact_course_id_fkey FOREIGN KEY (course_id) REFERENCES courses_dim (course_id);
ALTER TABLE course_skill_dim ADD CONSTRAINT course_skill_dim_course_id_fkey FOREIGN KEY (course_id) REFERENCES course_details_fact (course_id);
ALTER TABLE course_skill_dim ADD CONSTRAINT course_skill_dim_skill_id_fkey FOREIGN KEY (skill_id) REFERENCES skills_dim (skill_id);
"""

# Execute the SQL script
try:
    with engine.connect() as conn:
        conn.execute(text(alter_table_script))
    print("Primary and foreign keys set successfully.")
except Exception as e:
    print(f"An error occurred: {e}")

Primary and foreign keys set successfully.
